In [2]:
import pandas as pd
import os
from itertools import chain
import altair as alt
import numpy as np
#!pip install "vegafusion[embed]>=1.4.0"
#!pip install "vl-convert-python>=1.1.0"
#alt.data_transformers.enable("vegafusion")
#alt.data_transformers.disable_max_rows()
#alt.data_transformers.enable('csv')
#print("Active transformer:", alt.data_transformers.active)

notebook_dir = os.getcwd()
results_path = os.path.normpath(os.path.join(notebook_dir, "outputs", "results.csv"))
norms_path   = os.path.normpath(os.path.join(notebook_dir, "data", "all_norms_16variants_L_with_dnf.csv"))

# Load CSVs
results_df = pd.read_csv(results_path)
norms_df   = pd.read_csv(norms_path, dtype={"8bit_vector": str, "4bit_orig": str})

# --- Helpers to flatten ---
def flatten_ebsn_to_str(ebsn):
    flat_list = list(chain.from_iterable(chain.from_iterable(ebsn)))
    return ''.join(str(int(b)) for b in flat_list)

def flatten_base_sn_to_str(base_sn):
    flat_list = list(chain.from_iterable(base_sn))
    return ''.join(str(int(b)) for b in flat_list)

# Flatten columns in results
results_df['8bit_vector'] = results_df['eb_social_norm'].apply(eval).apply(flatten_ebsn_to_str)
results_df['4bit_orig']   = results_df['base_social_norm'].apply(eval).apply(flatten_base_sn_to_str)

# Merge and include DNF columns
merged_df = pd.merge(
    results_df,
    norms_df[["norm", "norm_desc", "variant_id", "8bit_vector", "4bit_orig",
              "Emotion_Leniency", "DNF", "DNF_literals"]],
    on=["8bit_vector", "4bit_orig"],
    how="left"
)

# Ensure numeric
merged_df["DNF_literals"] = pd.to_numeric(merged_df["DNF_literals"], errors="coerce")

In [13]:

# Pick norm
chosen_norm = "ImageScoring"
norm_df = merged_df[merged_df.norm == chosen_norm].copy()
norm_df["is_base"] = norm_df["variant_id"].str.endswith("_v1")

# If your results use a different name (e.g., "gamma_gaussian_n"),
# rename it once so plots are consistent:
if "gamma_center" not in norm_df.columns and "gamma_gaussian_n" in norm_df.columns:
    norm_df = norm_df.rename(columns={"gamma_gaussian_n": "gamma_center"})

# Aggregate per variant & gamma, carry DNF info (constant per variant)
agg_df = (
    norm_df
    .groupby(["variant_id", "DNF_literals", "Emotion_Leniency", "gamma_center", "is_base"], as_index=False)
    .agg(
        avg_coop=("average_cooperation", "mean"),
        std_coop=("average_cooperation", "std"),
        DNF=("DNF", "first")
    )
    .sort_values(["variant_id", "gamma_center"])
)

agg_df

,variant_id,DNF_literals,Emotion_Leniency,gamma_center,is_base,avg_coop,std_coop,DNF
0,ImageScoring_v1,1,1.00,0.0,True,30.18688,6.514240,A
1,ImageScoring_v1,1,1.00,0.1,True,27.97364,6.456988,A
2,ImageScoring_v1,1,1.00,0.2,True,30.59290,5.751677,A
3,ImageScoring_v1,1,1.00,0.3,True,29.19862,5.880590,A
4,ImageScoring_v1,1,1.00,0.4,True,29.07670,6.174220,A
...,...,...,...,...,...,...,...,...
171,ImageScoring_v9,3,0.75,0.6,False,18.19726,11.907804,A | (~E & ~R)
172,ImageScoring_v9,3,0.75,0.7,False,20.85058,17.518247,A | (~E & ~R)
173,ImageScoring_v9,3,0.75,0.8,False,20.16592,17.675892,A | (~E & ~R)
174,ImageScoring_v9,3,0.75,0.9,False,28.49390,24.706653,A | (~E & ~R)


In [12]:
# Make a percent-friendly copy
plot_df = agg_df.copy()

# If avg_coop is in [0,1], convert to %; if already 0–100, keep as-is
def to_percent(col):
    arr = col.to_numpy(dtype=float)
    # heuristic: if most values ≤ 1, treat as proportions
    needs_scale = (np.nanmean(arr <= 1.0) > 0.5)
    return arr * 100.0 if needs_scale else arr

plot_df["avg_coop_percent"] = to_percent(plot_df["avg_coop"])
plot_df["std_coop_percent"] = to_percent(plot_df["std_coop"])

# Base lines (no selections; color by DNF_literals; dashed if base)
line = alt.Chart(plot_df).mark_line().encode(
    x=alt.X("gamma_center:Q", title="Gamma value"),
    y=alt.Y("avg_coop_percent:Q",
            title="Average Cooperation (%)",
            scale=alt.Scale(domain=[0, 100])),
    color=alt.Color("Emotion_Leniency:O",
                    title="Emotion Leniency",
                    scale=alt.Scale(scheme="bluepurple")),
    strokeDash=alt.condition(
        alt.datum.is_base,
        alt.value([5, 5]),    # dashed for base
        alt.value([1, 0])     # solid otherwise
    ),
    strokeWidth=alt.condition(
        alt.datum.is_base,
        alt.value(8),
        alt.value(2)
    ),
    tooltip=[
        alt.Tooltip("variant_id:N", title="Variant"),
        alt.Tooltip("gamma_center:Q", title="Gamma"),
        alt.Tooltip("avg_coop_percent:Q", title="Mean coop (%)", format=".1f"),
        alt.Tooltip("std_coop_percent:Q", title="Std (%)", format=".1f"),
        alt.Tooltip("Emotion_Leniency:O", title="Leniency"),
        alt.Tooltip("is_base:N", title="Base?")
    ],
    detail="variant_id:N"
)

# Optional: end-of-line labels (still no interactivity)
endpoints = (
    plot_df.sort_values(["variant_id", "gamma_center"])
           .groupby("variant_id", as_index=False)
           .tail(1)
)

labels = alt.Chart(endpoints).mark_text(
    dx=5, align="left", baseline="middle"
).encode(
    x="gamma_center:Q",
    y=alt.Y("avg_coop_percent:Q",
            scale=alt.Scale(domain=[0, 100])),
    text="variant_id:N"
)

chart = (line + labels).properties(
    width=700, height=420,
    title=f"Performance of {chosen_norm} Variants (color = Emotion Leniency; dashed = base)"
)

chart

alt.LayerChart(...)

In [43]:
import numpy as np
import pandas as pd
import altair as alt

chosen_norm = "Shunning"

# 1) Filter + unify gamma column name
norm_df = merged_df[merged_df.norm == chosen_norm].copy()
if "gamma_center" not in norm_df.columns and "gamma_gaussian_n" in norm_df.columns:
    norm_df = norm_df.rename(columns={"gamma_gaussian_n": "gamma_center"})

norm_df = norm_df[norm_df.gamma_center==0.1]

# 2) Pre-aggregate: mean/std/count by (gamma_center, DNF_literals)
agg = (
    norm_df.groupby(["gamma_center", "DNF_literals"], as_index=False)
           .agg(mean_coop=("average_cooperation", "mean"),
                sd_coop=("average_cooperation", "std"),
                n=("variant_id", "nunique"))
)
# Precompute errorbar bounds
agg["lower"] = agg["mean_coop"] - agg["sd_coop"]
agg["upper"] = agg["mean_coop"] + agg["sd_coop"]


bars = alt.Chart(agg).mark_bar().encode(
    x=alt.X("DNF_literals:O", title="# literals (DNF)"),
    y=alt.Y("mean_coop:Q", title="Average cooperation", scale=alt.Scale(domain=[0, 100])),
    tooltip=[
        alt.Tooltip("DNF_literals:O", title="# literals"),
        alt.Tooltip("mean_coop:Q", title="mean coop"),
        alt.Tooltip("sd_coop:Q", title="std"),
        alt.Tooltip("n:Q", title="# variants")
    ]
)

err = base.mark_errorbar().encode(
    x="DNF_literals:O",
    y="lower:Q",
    y2="upper:Q"
)

chart = (bars + err).properties(
    width=600,
    title=f"{chosen_norm} — average cooperation by DNF literals (pick γ)"
)

chart

alt.LayerChart(...)

In [15]:
base

SchemaValidationError: '{'data': {'url': 'altair-data-342140fd4fa54b93a3397d3004376e61.csv', 'format': {'type': 'csv'}}, 'params': [{'name': 'gamma', 'bind': {'input': 'select', 'options': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], 'name': 'γ: '}, 'value': 0.0}], 'transform': [{'filter': '(datum.gamma_center === gamma)'}]}' is an invalid value.

'mark' is a required property

alt.Chart(...)

In [23]:
import pandas as pd
import numpy as np
import altair as alt

chosen_norm = "ImageScoring"

# Filter and unify gamma column name
norm_df = merged_df[merged_df.norm == chosen_norm].copy()
if "gamma_center" not in norm_df.columns and "gamma_gaussian_n" in norm_df.columns:
    norm_df = norm_df.rename(columns={"gamma_gaussian_n": "gamma_center"})

# Pre-aggregate: mean/std/count per (gamma, DNF_literals)
agg = (
    norm_df.groupby(["gamma_center", "DNF_literals"], as_index=False)
           .agg(mean_coop=("average_cooperation", "mean"),
                sd_coop=("average_cooperation", "std"),
                n=("variant_id", "nunique"))
)

agg["DNF_literals"] = agg["DNF_literals"].astype(int)
# If your cooperation is in 0..1 and you prefer %, uncomment:
# agg["mean_coop"] *= 100

heat = alt.Chart(agg).mark_rect().encode(
    x=alt.X("gamma_center:O", title="γ"),
    y=alt.Y("DNF_literals:O", title="DNF literals", sort="descending"),
    color=alt.Color("mean_coop:Q", title="Mean cooperation",
                    scale=alt.Scale(scheme="viridis", domain=[0,100])),
    tooltip=[
        alt.Tooltip("gamma_center:O", title="γ"),
        alt.Tooltip("DNF_literals:O", title="# literals"),
        alt.Tooltip("mean_coop:Q", title="mean coop", format=".3f"),
        alt.Tooltip("sd_coop:Q", title="std", format=".3f"),
        alt.Tooltip("n:Q", title="# variants")
    ]
).properties(
    width=400, height=350,
    title=f"{chosen_norm} — Cooperation by γ × DNF complexity"
)

heat


alt.Chart(...)

In [37]:
import pandas as pd
import numpy as np
import altair as alt

chosen_norm = "ImageScoring"

# Filter and unify gamma column name
norm_df = merged_df[merged_df.norm == chosen_norm].copy()
if "gamma_center" not in norm_df.columns and "gamma_gaussian_n" in norm_df.columns:
    norm_df = norm_df.rename(columns={"gamma_gaussian_n": "gamma_center"})

# Pre-aggregate: mean/std/count per (gamma, DNF_literals)
agg = (
    norm_df.groupby(["gamma_center", "Emotion_Leniency"], as_index=False)
           .agg(mean_coop=("average_cooperation", "mean"),
                sd_coop=("average_cooperation", "std"),
                n=("variant_id", "nunique"))
)

#agg["Emotion_Leniency"] = agg["Emotion_Leniency"].astype(int)
# If your cooperation is in 0..1 and you prefer %, uncomment:
# agg["mean_coop"] *= 100

heat = alt.Chart(agg).mark_rect().encode(
    x=alt.X("gamma_center:O", title="γ"),
    y=alt.Y("Emotion_Leniency:O", title="Emotion_Leniency", sort="descending"),
    color=alt.Color("mean_coop:Q", title="Mean cooperation",
                    scale=alt.Scale(scheme="viridis", domain=[0,100])),
    tooltip=[
        alt.Tooltip("gamma_center:O", title="γ"),
        alt.Tooltip("Emotion_Leniency:O", title="# literals"),
        alt.Tooltip("mean_coop:Q", title="mean coop", format=".3f"),
        alt.Tooltip("sd_coop:Q", title="std", format=".3f"),
        alt.Tooltip("n:Q", title="# variants")
    ]
).properties(
    width=400, height=350,
    title=f"{chosen_norm} — Cooperation by γ × Emotion Leniency"
)

heat


alt.Chart(...)

In [33]:
norm_df.Emotion_Leniency.value_counts()

Emotion_Leniency
0.50    3300
0.25    2200
0.75    2200
0.00     550
1.00     548
Name: count, dtype: int64